Для того, чтобы посчитать смены, следующий алгоритм

Есть соответсвие между операцией, станком, количеству ресурсов

Инфраструктура определяет, сколько одновременно можно сделать операций

И люди тоже привязаны к инфраструктуре

Поэтому надо сделать следующий файл с конфигурацией:

Станок Количество_станков Операция Тип_людей Количество_людей 

Останется только перевести из нормо-часов в дни.

1) Сначала заполняем по максимому дневные смены
2) при нехватке в месяц дневных смен, заполняем ночные 

Как будто это можно сделать средствами пандаса

1. Берём мерджим таблицу с операциями и таблицу конфигурации
2. Нормо-часы делим на количество станков
3. Дублируем каждую строчку со станком в соответсввии с количеством и дропаем количество
4. Получаем остаток от деления количества нормо-часов на 12, записываем это в колонку num_shifts
5. Делаем табличку со списком требуемых сотрудников
6. Прибавляем к num_shifts дату старта, тем самым получая с какого числа по какое будет дневная смена
7. Для тех, кто num_shifts > max_days, прибавляем  к дате старта (num_days - num_shift) (надо учесть, что может быть большой разрыв)

Этот алгоритм верен при параллельных операциях

На деле происходит последовательные операции. Глупо будет думать, что вот они сначала сделают все детали B, чтобы сделать деталь А. Наверное, сначала сделают n деталей В, потом их передадут дальше. По принципу конвейера, это просто даст некотурую начальную задерджку. Возмонжно, она будет в рамках часа, не более. Поэтому такой алгоритм может дать приблизительную оценку.